batch normalization 有利于训练深层神经网络，持续增加深层网络的收敛速度  
猜想：在反向传播的过程中，由于网络的深度，导致了靠近输出层的梯度比较大（因为LOSS原始数据也大），靠近输入的层梯度越小。  
当lr一定时，输入层因为梯度比较小，更新更少，但是输出层的梯度是很大的，所以输出层会很快收敛。  
但是如果输入层的参数更新了之后（因为不同的batch里面的数据的分布可能不同），输出层的参数又要重新学习（以适应不同的分布），导致网络整体的参数收敛的速度变慢。  
> 需求：在更换batch，输入层参数更新的时候避免更改输出层的参数  
解决方法：由于我们本质上就是在学习数据的分布，所以如果能够把每一个batch的数据都能固定下来，就能更容易学习不同batches之间的细微变化  
* 固定下小批量的均值和方差
$$ {\mu }_{B}=\frac {1} {|B|}\sum _{i\in B} {{x}_{i}}\, ,\, {\sigma }^{2}_{B}=\frac {1} {|B|}\sum _{i\in B} {{({x}_{i}-{\mu }_{B})}^{2}}+\epsilon  $$  

* 加上可学习的参数构成最终的层：  
$$ {x}_{i+1}=\gamma \frac {{x}_{i}-{\mu }_{B}} {{\sigma }_{B}}+\beta  $$  
可学习的参数: $ \gamma $、$ \beta $,NB层作用在全联接和卷积的输出上、激活函数之前，对于全联接层，作用在特征维，对于卷积层，作用在通道维（当作是卷积层的一个特征维）

有论文指出，这个东西可能就是在以恶搞小批量里面增加噪音（随机的一个均值和方差）来控制模型的复杂度，没必要和DropOut混用

In [1]:
import torch
from torch import nn
from d2l import torch as d2l


def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过is_grad_enabled来判断当前模式是训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)    #做预测的话就用全局的mean和方差来归一化推理值
    else:
        assert len(X.shape) in (2, 4)   
        #要么2，要么4，shape==2就是全联接、4就是卷积层,否则就让它中断掉
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差，均是行向量
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
            # 这里我们需要保持X的形状以便后面可以做广播运算
            mean = X.mean(dim=(0, 2, 3), keepdim=True)  
            #求出来的就是一个1,n,1,1的一个4D的东西
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean  #在尽量参考现在当前层计算出的mean来更新全局的mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data

In [ ]:
class BatchNorm(nn.Module):
    # num_features：完全连接层的输出数量或卷积层的输出通道数。
    # num_dims：2表示完全连接层，4表示卷积层
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), BatchNorm(6, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), BatchNorm(16, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(16*4*4, 120), BatchNorm(120, num_dims=2), nn.Sigmoid(),
    nn.Linear(120, 84), BatchNorm(84, num_dims=2), nn.Sigmoid(),
    nn.Linear(84, 10))

In [ ]:
lr, num_epochs, batch_size = 1.0, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())